In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score as bas
import gc
gc.enable()

from sklearn.linear_model import LogisticRegression, SGDClassifier

from os import listdir
from os.path import isfile, join

In [2]:
data = pd.read_csv('../data-reduced-800-v3-shuffled.csv', index_col = 0)
testfile = pd.read_csv('../test.csv')
catcode = pd.read_csv('../data-simplified-1-catcode.csv', header = None, names = ['category'])['category']

/store/tveiga/miniconda3/envs/py/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
sp = int(len(data) * 0.8) # Split Point
sp2 = len(data)
y_train = data.category.values[:sp]
y_test = data.category.values[sp:sp2]
rel = data.label_quality.values[sp:sp2]

## Val

In [4]:
DIR = '../ensemb3/'

In [5]:
from os import listdir
from os.path import isfile, join
val = {f[4:-4] : f for f in listdir(DIR) if ((isfile(join(DIR, f))) and f[0] == 'v')}

In [6]:
val

{'sgd_char-v7': 'val_sgd_char-v7.csv',
 'lstm_word': 'val_lstm_word.csv',
 'lstm_char_unproc': 'val_lstm_char_unproc.csv',
 'mnb': 'val_mnb.csv',
 'sgd_word-v7': 'val_sgd_word-v7.csv',
 'sgd_char-v7_45': 'val_sgd_char-v7_45.csv'}

In [9]:
%%time
for k, v in val.items():
    print(k)
    val[k] = pd.read_csv(DIR+v, dtype = np.float32, header = None, index_col = False)

sgd_char-v7
lstm_word
lstm_char_unproc
mnb
sgd_word-v7
sgd_char-v7_45
CPU times: user 5min 18s, sys: 12.3 s, total: 5min 30s
Wall time: 6min 47s


In [11]:
classes = pd.Series(range(1588))
out = classes.loc[~classes.isin(pd.Series(y_test[rel == 0]))]
outsiders = pd.Series(y_test).isin(out)

In [12]:
num_rels = (rel == 0).sum()
num_rels, len(rel)

(90465, 275165)

In [13]:
dfs = sorted(val.keys() - {})

In [14]:
X = np.zeros((len(y_test) * 1588, len(dfs)), dtype = np.float32)
y = np.zeros((len(X)), dtype = np.int32)

In [15]:
val_len = len(y_test)

In [16]:
for k, v in val.items():
    print(k, v.shape)

sgd_char-v7 (275165, 1588)
lstm_word (275165, 1588)
lstm_char_unproc (275165, 1588)
mnb (275165, 1588)
sgd_word-v7 (275165, 1588)
sgd_char-v7_45 (275165, 1588)


In [17]:
# %%time
mask = rel == 0
for i in range(1588):
    for d, df in enumerate(dfs):
        X[i * val_len : (i + 1) * val_len, d] = val[df].values[:, i]
        y[i * val_len : (i + 1) * val_len] = 1 * (y_test == i)
        
#         X[i * val_len : (i + 1) * val_len, d] = val[df].values[mask, i]
#         y[i * val_len : (i + 1) * val_len] = 1 * (y_test[mask] == i)

In [24]:
ix = np.random.randint(0, len(X), len(X) // 100) # better than shuffle arange ??? how?

In [25]:
%%time
est = SGDClassifier(loss = 'modified_huber', max_iter=10, tol =  1e-5, penalty='l2', alpha = 1e-4)
est.fit(X[ix], y[ix]) # consider sample weight?

CPU times: user 5.56 s, sys: 47.9 ms, total: 5.61 s
Wall time: 5.01 s


In [26]:
%%time
# 0.916317 (10)
new = sum(val[df].values[mask] * coef for df, coef in zip(dfs, est.coef_.flatten())) / est.coef_.sum()
val_pred = new.argmax(axis = 1)
print(bas(y_test[rel == 0], val_pred))

0.9170534864774611
CPU times: user 9.56 s, sys: 737 ms, total: 10.3 s
Wall time: 4.96 s


/store/tveiga/miniconda3/envs/py/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [27]:
# 0.9170534864774611
for df, coef in zip(dfs, est.coef_.flatten()):
    print(f"'{df}' : {coef},")

'lstm_char_unproc' : 0.15214980219793464,
'lstm_word' : 0.5057028723619695,
'mnb' : 0.45090371936039003,
'sgd_char-v7' : 0.32103481728540106,
'sgd_char-v7_45' : 0.35221132620470513,
'sgd_word-v7' : 0.39905860580347985,


In [13]:
# These values were obtained in the cell above
coefval = {
'lstm_char_unproc' : 0.15214980219793464,
'lstm_word' : 0.5057028723619695,
'mnb' : 0.45090371936039003,
'sgd_char-v7' : 0.32103481728540106,
'sgd_char-v7_45' : 0.35221132620470513,
'sgd_word-v7' : 0.39905860580347985,
}

In [14]:
%%time
ensemble_val = sum(val[df] * coef for df, coef in coefval.items()) / sum(coefval.values())

CPU times: user 33.9 s, sys: 17.7 s, total: 51.6 s
Wall time: 11.1 s


In [15]:
%%time
ensemble_val.to_csv('../ensens/val_ensemble11.csv', index = False, header = False)

CPU times: user 7min 21s, sys: 9.85 s, total: 7min 30s
Wall time: 7min 38s


## Test

In [30]:
DIR = '../ensemb3/'

In [31]:
test = {f[5:-4] : f for f in listdir(DIR) if ((isfile(join(DIR, f))) and f[0] == 't')}

In [32]:
test, len(test)

({'lstm_word': 'test_lstm_word.csv',
  'lstm_char_unproc': 'test_lstm_char_unproc.csv',
  'sgd_char-v7_45': 'test_sgd_char-v7_45.csv',
  'sgd_word-v7': 'test_sgd_word-v7.csv',
  'mnb': 'test_mnb.csv',
  'sgd_char-v7': 'test_sgd_char-v7.csv'},
 6)

In [35]:
%%time
for k, v in test.items():
    print(k)
    test[k] = pd.read_csv(DIR+v, header = None, index_col = False)

lstm_word
lstm_char_unproc
sgd_char-v7_45
sgd_word-v7
mnb
sgd_char-v7
CPU times: user 5min 20s, sys: 13.7 s, total: 5min 34s
Wall time: 6min 41s


In [36]:
# These values were obtained in the section above
coeftest = {
'lstm_char_unproc' : 0.15214980219793464,
'lstm_word' : 0.5057028723619695,
'mnb' : 0.45090371936039003,
'sgd_char-v7' : 0.32103481728540106,
'sgd_char-v7_45' : 0.35221132620470513,
'sgd_word-v7' : 0.39905860580347985,
}

In [37]:
%%time
ensemble = sum(test[df] * coef for df, coef in coeftest.items()) / sum(coeftest.values())

CPU times: user 35.6 s, sys: 11.5 s, total: 47.1 s
Wall time: 16.5 s


In [38]:
sample = pd.read_csv('../sample_submission.csv')

In [39]:
%%time
sample['category'] = np.vectorize(catcode.get)(ensemble.values.argmax(axis=1))

CPU times: user 3.37 s, sys: 569 ms, total: 3.94 s
Wall time: 3.84 s


In [40]:
sample.head()

,id,category
0,0,DIAPER_BAGS
1,1,BABY_CHANGING_PADS
2,2,ENGINE_COOLING_FAN_MOTORS
3,3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,BABY_CAR_SEATS


In [42]:
sample.to_csv('../submission_ensemble11.csv', index = False)
sample.shape

(246955, 2)

In [43]:
pd.read_csv('../submission_ensemble11.csv').head()

,id,category
0,0,DIAPER_BAGS
1,1,BABY_CHANGING_PADS
2,2,ENGINE_COOLING_FAN_MOTORS
3,3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,BABY_CAR_SEATS


In [44]:
%%time
ensemble.to_csv('../ensens/test_ensemble11.csv', index = False, header = False)

CPU times: user 6min 30s, sys: 12.3 s, total: 6min 42s
Wall time: 6min 50s
